<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: C:\Users\Halfdan\Disk_L\code\Python\dl-homework\archive.zip

  0%|          | 0.00/84.9M [00:00<?, ?B/s]
  1%|          | 524k/84.9M [00:00<00:54, 1.54MB/s]
  1%|1         | 1.05M/84.9M [00:00<00:33, 2.50MB/s]
  3%|3         | 2.62M/84.9M [00:00<00:13, 6.08MB/s]
  8%|8         | 6.82M/84.9M [00:00<00:04, 15.8MB/s]
 19%|#8        | 15.7M/84.9M [00:00<00:02, 30.5MB/s]
 23%|##2       | 19.4M/84.9M [00:01<00:03, 20.9MB/s]
 30%|###       | 25.7M/84.9M [00:01<00:02, 28.7MB/s]
 38%|###7      | 32.0M/84.9M [00:01<00:01, 32.2MB/s]
 43%|####2     | 36.2M/84.9M [00:01<00:01, 30.0MB/s]
 50%|#####     | 42.5M/84.9M [00:01<00:01, 36.7MB/s]
 57%|#####6    | 48.2M/84.9M [00:01<00:00, 41.5MB/s]
 65%|######5   | 55.6M/84.9M [00:01<00:00, 49.3MB/s]
 75%|#######4  | 63.4M/84.9M [00:01<00:00, 56.4MB/s]
 85%|########5 | 72.4M/84.9M [00:02<00:00, 64.7MB/s]
 96%|#########5| 81.3M/84.9M [00:02<00:00, 71.0MB/s]
100%|######

Импортируем нужные библиотеки:

In [1]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [2]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [4]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [5]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [6]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [7]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [8]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [9]:
word2vec = api.load("word2vec-google-news-300")

In [10]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [11]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [12]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        return [w for w in tokenizer.tokenize(text) if all(c not in string.punctuation for c in w) and len(w) > 3]

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [13]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [14]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [15]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])

In [16]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [17]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1004', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [19]:
from torch.utils.data import DataLoader


batch_size = 4096
num_workers = 0

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [[b["target"]] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [20]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели
        optimizer.zero_grad()
        preds = model(features)
        loss = criterion(preds, targets) # Посчитай лосс
        # Обнови параметры модели
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            preds = model(features)
            loss = criterion(preds, targets) # Посчитай лосс
            acc = torch.sum(((preds > 0.5).float() == targets)) / len(features) # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [22]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 10

# Возьмем небольшую модель с 1 скрытым слоем, чтобы сильно не грузить gpu.
model = nn.Sequential(
    nn.Linear(vector_size, 128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)
model = model.cuda()
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=lr)

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

**Ответ.** Выберем в качестве метрики точность, поскольку мы решаем задачу классификации. Метрика лосс показывает, насколько модель уверена в своих предсказаниях, что нас не слишком сильно интересует. 

In [23]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Acc"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Acc"]

Epoch 1. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.5205153037631323, 'Test Acc': 0.7389594184027778}


Epoch 2. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.4974411664501069, 'Test Acc': 0.7549641927083334}


Epoch 10. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.49757023699699887, 'Test Acc': 0.7543751550099206}


In [24]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))


print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/79 [00:00<?, ?it/s]

{'Test Loss': 0.5210455354255966, 'Test Acc': 0.7398480765427216}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

## Перый эксперимент.

In [25]:
class TwitterDatasetContext(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, window: int):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        self.window = window

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        return [w for w in tokenizer.tokenize(text) if all(c not in string.punctuation for c in w) and len(w) > 3]

    def get_embeddings_(self, tokens):
        embeddings = []
        for i in range(len(tokens)):
            if tokens[i] in self.word2vec:
                embedding = (self.word2vec.get_vector(tokens[i]) - mean) / std
            else:
                embedding = np.zeros((self.word2vec.vector_size))
                for j in range(1, self.window + 1):
                    if i - j >= 0:
                        embedding += (self.word2vec.get_vector(tokens[i - j]) - mean) / std if tokens[i - j] in self.word2vec else np.zeros((self.word2vec.vector_size))
                    if i + j < len(tokens):
                        embedding += (self.word2vec.get_vector(tokens[i + j]) - mean) / std if tokens[i + j] in self.word2vec else np.zeros((self.word2vec.vector_size))
            embeddings.append(embedding)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [26]:
# будем просматривать по 2 слова слева и справа
dev = TwitterDatasetContext(dev_data, "text", "emotion", word2vec, 2)

In [27]:
train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [28]:
model = nn.Sequential(
    nn.Linear(vector_size, 128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)
model = model.cuda()
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=lr)

In [29]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Acc"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Acc"]

Epoch 1. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.5294714456512815, 'Test Acc': 0.7324451264880952}


Epoch 2. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.5063535187925611, 'Test Acc': 0.7497364831349206}


Epoch 10. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.5054233594546242, 'Test Acc': 0.7498643663194444}


In [30]:
test_loader = DataLoader(
    TwitterDatasetContext(test_data, "text", "emotion", word2vec, 2), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False,
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))


print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/79 [00:00<?, ?it/s]

{'Test Loss': 0.5269827895526644, 'Test Acc': 0.7346902195411392}


## Второй эксперимент.

In [31]:
from sparsesvd import sparsesvd
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
# Извлечем векторные представления слов в тренировочной выборке. Далее будем использовать их для тренировки и валидации.
targets = dev_data["text"]
corpus = []
tokenizer = nltk.WordPunctTokenizer()
for i in range(len(targets)):
    tokens = [w for w in tokenizer.tokenize(targets[i]) if all(c not in string.punctuation for c in w) and len(w) > 3]
    corpus.append(" ".join(tokens))
vectorizer = TfidfVectorizer(min_df=5)
X = vectorizer.fit_transform(corpus)
_, _, vt = sparsesvd(X.tocsc(), 300)
k, v = vectorizer.get_feature_names(), vt.T
tfid = {k[i]: v[i] for i in range(len(v))}                  

In [33]:
class TwitterDatasetTfid(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, tfid: dict):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        self.tfid = tfid

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        return [w for w in tokenizer.tokenize(text) if all(c not in string.punctuation for c in w) and len(w) > 3]

    def get_embeddings_(self, tokens):
        embeddings = []
        for i in range(len(tokens)):
            if tokens[i] in self.word2vec:
                embedding = (self.word2vec.get_vector(tokens[i]) - self.mean) / self.std
                embedding += self.tfid[tokens[i]] if tokens[i] in self.tfid else np.zeros((self.word2vec.vector_size))
            else:
                embedding = self.tfid[tokens[i]] if tokens[i] in self.tfid else np.zeros((self.word2vec.vector_size))
            embeddings.append(embedding)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [34]:
dev = TwitterDatasetTfid(dev_data, "text", "emotion", word2vec, tfid)

In [35]:
train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [36]:
model = nn.Sequential(
    nn.Linear(vector_size, 128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)
model = model.cuda()
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=lr)

In [37]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Acc"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Acc"]

Epoch 1. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.520779101621537, 'Test Acc': 0.7381262400793651}


Epoch 2. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.4987496833006541, 'Test Acc': 0.7545999193948413}


Epoch 10. Train Loss: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/63 [00:00<?, ?it/s]

{'Test Loss': 0.49861302971839905, 'Test Acc': 0.7536117311507936}


In [38]:
test_loader = DataLoader(
    TwitterDatasetTfid(test_data, "text", "emotion", word2vec, tfid), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))


print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/79 [00:00<?, ?it/s]

{'Test Loss': 0.5189068430586706, 'Test Acc': 0.7407381081882911}


# Вывод.

Метрики моделей на валидации получились такими:

In [42]:
dict(
    original_model="'Test Loss': 0.5210455354255966, 'Test Acc': 0.7398480765427216",
    context_model="'Test Loss': 0.5269827895526644, 'Test Acc': 0.7346902195411392",
    tfid_model="'Test Loss': 0.5189068430586706, 'Test Acc': 0.7407381081882911"
)

{'original_model': "'Test Loss': 0.5210455354255966, 'Test Acc': 0.7398480765427216",
 'context_model': "'Test Loss': 0.5269827895526644, 'Test Acc': 0.7346902195411392",
 'tfid_model': "'Test Loss': 0.5189068430586706, 'Test Acc': 0.7407381081882911"}

Добавление контекста ухудшило результат. Здесь возможно несколько причин. Первая - выбран недостаточный размер окна. Вторая - изначально плохой способ кодирования предложения путем сложения векторов токенов усугубился дополнительным преплюсовыванием векторов из контекста. 

Добавление tfid положительно повлияло на качество модели. Скорее всего это можно объяснить тем, что методы векторизации смотрят по разному на слова, и сумма их эмбеддингов достаточно хорошо кодирует слова в целом.